In [21]:
import numpy as np
import sys

sys.path.append("..")

from src.auth.aurora import get_portal_client

In [2]:
portal_client = get_portal_client()

In [45]:
services_df = portal_client.query_to_pandas(
    """SELECT 
        tests.mode_change_id,
        tests.visit_id,
        SUM(
            CASE
            WHEN tests.tested="P"
            AND NOT tests.advisory
            THEN 1
            ELSE 0
            END
        ) AS passed,
        SUM(
            CASE
            WHEN tests.tested="P"
            AND tests.advisory
            THEN 1
            ELSE 0
            END
        ) AS passed_with_advisory,
        SUM(
            CASE
            WHEN tests.tested="F"
            THEN 1
            ELSE 0
            END
        ) AS failed
    FROM topaz.api_service_tests AS tests
    WHERE id >= 9559055
    GROUP BY tests.mode_change_id, tests.visit_id
    """
)

In [46]:
assert np.all(
    (
        ~services_df[
            ["mode_change_id", "visit_id"]
        ].isna()
    ).sum(axis=1)
    == 1
)

In [47]:
services_df["service_id"] = services_df[[
    "mode_change_id", "visit_id"
]].bfill(axis=1).iloc[:, 0]

In [48]:
services_df

,mode_change_id,visit_id,passed,passed_with_advisory,failed,service_id
0,NaN,105111.0,11,0,0,105111.0
1,NaN,105107.0,83,25,0,105107.0
2,NaN,105105.0,6,0,0,105105.0
3,NaN,105058.0,24,0,0,105058.0
4,723036.0,NaN,191,0,0,723036.0
...,...,...,...,...,...,...
105654,1009793.0,NaN,0,1,0,1009793.0
105655,1009864.0,NaN,1,0,0,1009864.0
105656,1009865.0,NaN,1,0,0,1009865.0
105657,1010030.0,NaN,1,0,0,1010030.0


In [50]:
grouped_df = services_df.groupby("service_id").agg(
    {
        "passed": "sum",
        "passed_with_advisory": "sum",
        "failed": "sum"
    }
)
grouped_df = grouped_df[grouped_df.sum(axis=1) > 0]

In [54]:
grouped_df["passed"] / grouped_df.sum(axis=1)

service_id
64816.0        1
87295.0        1
87482.0        1
90133.0        1
90596.0        1
              ..
176899527.0    1
176899528.0    1
176899529.0    1
176899530.0    1
176899532.0    1
Length: 105448, dtype: object

In [55]:
grouped_df["passed_with_advisory"] / grouped_df.sum(axis=1)

service_id
64816.0        0
87295.0        0
87482.0        0
90133.0        0
90596.0        0
              ..
176899527.0    0
176899528.0    0
176899529.0    0
176899530.0    0
176899532.0    0
Length: 105448, dtype: object

In [56]:
grouped_df["failed"] / grouped_df.sum(axis=1)

service_id
64816.0        0
87295.0        0
87482.0        0
90133.0        0
90596.0        0
              ..
176899527.0    0
176899528.0    0
176899529.0    0
176899530.0    0
176899532.0    0
Length: 105448, dtype: object